In [1]:
!git clone https://github.com/RichardStaszkiewicz/CNN-Hyperparameter-Tuning.git
!mv /content/CNN-Hyperparameter-Tuning/modules /content/modules
!mv /content/CNN-Hyperparameter-Tuning/model /content/model
!mkdir logs
!pip install "ray[tune]" torch torchvision pytorch-lightning

Cloning into 'CNN-Hyperparameter-Tuning'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 43 (delta 13), reused 33 (delta 6), pack-reused 0
Receiving objects: 100% (43/43), 94.01 KiB | 7.23 MiB/s, done.
Resolving deltas: 100% (13/13), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 59.5 MB/s eta 0:00:00


In [2]:
from modules import plmodules as plm
from ray import tune, air
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback
import pytorch_lightning as pl
import yaml

## Config

In [3]:
with open("/content/model/configs/model.yaml", 'r') as stream:
      default_config=yaml.safe_load(stream)

default_config = default_config['model']

In [7]:
# Grid search
# search_config = {
#     "batch_size": tune.grid_search([64, 128, 256]),
#     "lr": tune.grid_search([0.01, 0.1]),
#     "mlp_out_l0": tune.grid_search([64, 128]),
#     "mlp_af_l0": tune.grid_search(['relu', 'none']),
#     "mlp_bn_l0": tune.grid_search([True, False]),
#     "mlp_do_l0": tune.grid_search([0.1, 0.3])
# }
# Random search
search_config = {
    "batch_size": tune.choice([64, 128, 256]),
    "lr": tune.uniform(0.01, 0.1),
    "mlp_out_l0": tune.uniform(64, 128),
    "mlp_af_l0": tune.choice(['relu', 'None']),
    "mlp_bn_l0": tune.choice([True, False]),
    "mlp_do_l0": tune.uniform(0.095, 0.305)
}

In [8]:
config = default_config | search_config

## Scheduers

In [9]:
scheduler_asha = ASHAScheduler(
    time_attr="time_total_s",
    max_t=150,
    grace_period=30,
    reduction_factor=2,
)

## Callbacks

In [10]:
tune_report_callback = TuneReportCheckpointCallback(
    metrics={
    "ptl/train_loss": "ptl/train_loss",
    'ptl/train_accuracy': 'ptl/train_accuracy',
    'ptl/val_loss': 'ptl/val_loss',
    'ptl/val_accuracy': 'ptl/val_accuracy',
    'time_total_s': 'time_total_s',
    },
    filename="ray_ckpt",
    on="validation_end",
)

## Trainable

In [ ]:
def actualise_config(config):
  mlp = [int(k.replace("mlp_out_l", '')) for k in config.keys() if "mlp_out_l" in k]
  for l in mlp:
    config["mlp_config"]['block_list'][l]['out_size'] = config[f"mlp_out_l{l}"]
    config["mlp_config"]['block_list'][l+1]['in_size'] = config[f"mlp_out_l{l}"]
  mlp = [int(k.replace("mlp_af_l", '')) for k in config.keys() if "mlp_af_l" in k]
  for af in mlp:
    config["mlp_config"]['block_list'][af]['activation_fun'] = config[f"mlp_af_l{af}"]
  mlp = [int(k.replace("mlp_bn_l", '')) for k in config.keys() if "mlp_bn_l" in k]
  for bn in mlp:
    config["mlp_config"]['block_list'][bn]['batch_norm'] = config[f"mlp_bn_l{bn}"]
  mlp = [int(k.replace("mlp_do_l", '')) for k in config.keys() if "mlp_do_l" in k]
  for do in mlp:
    config["mlp_config"]['block_list'][do]['dropout'] = config[f"mlp_do_l{do}"]
  return config

In [ ]:
def run_with_tune(config, epochs=50):
    config = actualise_config(config)
    model = plm.MNISTClassifier(config)
    dm = plm.MNISTDataModule(config['batch_size'])
    trainer = pl.Trainer(
        max_epochs=epochs,
        fast_dev_run=False,
        callbacks=[
            tune_report_callback,
        ],
    )
    trainer.fit(model, dm)

## Reporter

In [ ]:
reporter = CLIReporter(
    parameter_columns=[search_config.keys()],
    metric_columns=[ "time_total_s", "ptl/train_accuracy", "ptl/val_loss", "ptl/val_accuracy", "training_iteration"]
)

## Trial

In [ ]:
train_fn_with_parameters = tune.with_parameters(run_with_tune, epochs=50)

In [ ]:
resources_per_trial = {"cpu": 2, "gpu": 1}

In [ ]:
tuner = tune.Tuner(
        tune.with_resources(
            train_fn_with_parameters,
            resources=resources_per_trial
        ),
        tune_config=tune.TuneConfig(
            metric="ptl/val_loss",
            mode="min",
            search_alg=tune.search.BasicVariantGenerator(),
            #time_budget_s=600,
            scheduler=scheduler_asha,
            #num_samples=-1,
        ),
        run_config=air.RunConfig(
            name="tune_mnist_asha",
            progress_reporter=reporter,
        ),
        param_space=config,
    )
results = tuner.fit()

2023-09-02 16:40:14,241	INFO worker.py:1621 -- Started a local Ray instance.
2023-09-02 16:40:16,715	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2023-09-02 16:40:16,720	INFO tune.py:666 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-09-02 16:40:17,149	WARNING tune.py:997 -- AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


+----------------------------------------------------------+
| Configuration for experiment     tune_mnist_asha         |
+----------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator   |
| Scheduler                        AsyncHyperBandScheduler |
| Number of trials                 96                      |
+----------------------------------------------------------+

View detailed results here: /root/ray_results/tune_mnist_asha
To visualize your results with TensorBoard, run: `tensorboard --logdir /root/ray_results/tune_mnist_asha`

Trial status: 16 PENDING
Current time: 2023-09-02 16:40:17. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs
+---------------------------------------------------------------------------------------------------------------------+
| Trial name                  status       batch_size     lr     mlp_out_l0   mlp_af_l0     mlp_bn_l0       mlp_do_l0 |
+------------------------------------

(run_with_tune pid=1086) GPU available: True (cuda), used: True
(run_with_tune pid=1086) TPU available: False, using: 0 TPU cores
(run_with_tune pid=1086) IPU available: False, using: 0 IPUs
(run_with_tune pid=1086) HPU available: False, using: 0 HPUs
(run_with_tune pid=1086) Missing logger folder: /root/ray_results/tune_mnist_asha/run_with_tune_62480_00000_0_batch_size=64,lr=0.0100,mlp_af_l0=relu,mlp_bn_l0=True,mlp_do_l0=0.1000,mlp_out_l0=64_2023-09-02_16-40-17/lightning_logs
(run_with_tune pid=1086) 2023-09-02 16:40:29.798559: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(run_with_tune pid=1086) To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(run_with_tune pid=1086) 2023-09-02 16:40:30.732033: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

(run_with_tune pid=1086) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(run_with_tune pid=1086) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to data/FashionMNIST/raw/train-images-idx3-ubyte.gz


 83%|████████▎ | 21856256/26421880 [00:02<00:00, 16035769.18it/s]


...

Epoch 1: 100%|██████████| 375/375 [00:29<00:00, 12.52it/s, v_num=0]
(run_with_tune pid=1086) 
                                                                        
Epoch 0:  43%|████▎     | 80/188 [00:05<00:07, 13.75it/s, v_num=0]
Trial status: 95 TERMINATED | 1 RUNNING
Current time: 2023-09-02 18:39:55. Total running time: 1hr 59min 38s
Logical resource usage: 2.0/2 CPUs, 1.0/1 GPUs
Current best trial: 62480_00000 with ptl/val_loss=0.4649209976196289 and params={'regularization_ratio': 0.5, 'batch_size': 64, 'lr': 0.01, 'resnet_config': {'first_conv': {'in_channels': 1, 'out_channels': 32, 'kernel_size': 3, 'stride': 2, 'padding': 1}, 'block_list': [{'in_channels': 32, 'out_channels': 16, 'kernel_size': 3, 'stride': 2, 'padding': 1}, {'in_channels': 16, 'out_channels': 8, 'kernel_size': 3, 'stride': 1, 'padding': 'same'}], 'pool_size': 2}, 'mlp_config': {'block_list': [{'in_size': 72, 'out_size': 64, 'activation_fun': 'relu', 'batch_norm': True, 'dropout': 0.1}, {'in_size': 64, 'ou

(run_with_tune pid=1086) Metric time_total_s does not exist in `trainer.callback_metrics.


Epoch 1: 100%|██████████| 188/188 [00:16<00:00, 11.70it/s, v_num=0]
(run_with_tune pid=1086) 
Validation: 0it [00:00, ?it/s]
(run_with_tune pid=1086) 
Validation DataLoader 0:   0%|          | 0/47 [00:00<?, ?it/s]
(run_with_tune pid=1086) 
Validation DataLoader 0:  43%|████▎     | 20/47 [00:01<00:01, 16.93it/s]
Trial status: 95 TERMINATED | 1 RUNNING
Current time: 2023-09-02 18:40:25. Total running time: 2hr 0min 8s
Logical resource usage: 2.0/2 CPUs, 1.0/1 GPUs
Current best trial: 62480_00000 with ptl/val_loss=0.4649209976196289 and params={'regularization_ratio': 0.5, 'batch_size': 64, 'lr': 0.01, 'resnet_config': {'first_conv': {'in_channels': 1, 'out_channels': 32, 'kernel_size': 3, 'stride': 2, 'padding': 1}, 'block_list': [{'in_channels': 32, 'out_channels': 16, 'kernel_size': 3, 'stride': 2, 'padding': 1}, {'in_channels': 16, 'out_channels': 8, 'kernel_size': 3, 'stride': 1, 'padding': 'same'}], 'pool_size': 2}, 'mlp_config': {'block_list': [{'in_size': 72, 'out_size': 64, 'act

(run_with_tune pid=1086) Metric time_total_s does not exist in `trainer.callback_metrics.


Epoch 1: 100%|██████████| 375/375 [08:03<00:00,  1.29s/it, v_num=0]
(run_with_tune pid=1086) 
Epoch 1: 100%|██████████| 188/188 [07:16<00:00,  2.32s/it, v_num=0]
Trial status: 95 TERMINATED | 1 RUNNING
Current time: 2023-09-02 18:40:55. Total running time: 2hr 0min 38s
Logical resource usage: 2.0/2 CPUs, 1.0/1 GPUs
Current best trial: 62480_00000 with ptl/val_loss=0.4649209976196289 and params={'regularization_ratio': 0.5, 'batch_size': 64, 'lr': 0.01, 'resnet_config': {'first_conv': {'in_channels': 1, 'out_channels': 32, 'kernel_size': 3, 'stride': 2, 'padding': 1}, 'block_list': [{'in_channels': 32, 'out_channels': 16, 'kernel_size': 3, 'stride': 2, 'padding': 1}, {'in_channels': 16, 'out_channels': 8, 'kernel_size': 3, 'stride': 1, 'padding': 'same'}], 'pool_size': 2}, 'mlp_config': {'block_list': [{'in_size': 72, 'out_size': 64, 'activation_fun': 'relu', 'batch_norm': True, 'dropout': 0.1}, {'in_size': 64, 'out_size': 10, 'activation_fun': 'logsoftmax', 'batch_norm': False, 'dropou

In [ ]:
# Trial status: 96 TERMINATED
# Current time: 2023-09-02 18:41:36. Total running time: 2hr 1min 18s
# Logical resource usage: 2.0/2 CPUs, 1.0/1 GPUs
df = results.get_dataframe()[['training_iteration', 'config/batch_size', 'config/lr', 'config/mlp_af_l0', 'config/mlp_bn_l0', 'config/mlp_do_l0', 'config/mlp_out_l0', 'ptl/val_loss', 'ptl/val_accuracy']]
df.sort_values('ptl/val_loss', ascending=True).head(20)

,training_iteration,config/batch_size,config/lr,config/mlp_af_l0,config/mlp_bn_l0,config/mlp_do_l0,config/mlp_out_l0,ptl/val_loss,ptl/val_accuracy
0,6,64,0.01,relu,True,0.1,64,0.464921,0.825733
49,7,128,0.01,relu,True,0.1,128,0.465987,0.826847
73,7,128,0.01,relu,True,0.3,128,0.471249,0.822388
26,8,256,0.01,relu,True,0.3,64,0.476863,0.819004
61,7,128,0.01,relu,False,0.1,128,0.478739,0.820178
60,6,64,0.01,relu,False,0.1,128,0.479928,0.820506
85,7,128,0.01,relu,False,0.3,128,0.491108,0.815976
12,5,64,0.01,relu,False,0.1,64,0.497068,0.813628
37,7,128,0.01,relu,False,0.3,64,0.500579,0.813494
84,6,64,0.01,relu,False,0.3,128,0.501533,0.813122
